## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-28-14-48-20 +0000,nyt,National Guard Soldier Dies After Being Shot i...,https://www.nytimes.com/2025/11/28/us/national...
1,2025-11-28-14-46-00 +0000,wsj,Who Is D.C. Shooting Suspect Rahmanullah Lakan...,https://www.wsj.com/politics/national-security...
2,2025-11-28-14-45-24 +0000,nyt,Death Toll Rises to 128 in Hong Kong High-Rise...,https://www.nytimes.com/2025/11/28/world/asia/...
3,2025-11-28-14-44-00 +0000,wsj,Dow Inches Up in Black Friday Trading,https://www.wsj.com/finance/stocks/global-stoc...
4,2025-11-28-14-43-00 +0000,wsj,Telefonica Scores Provisional Rights to Spain’...,https://www.wsj.com/business/retail/telefonica...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2373/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
99,trump,23
0,national,21
7,shooting,21
1,guard,20
18,fire,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
171,2025-11-28-00-57-00 +0000,nypost,Trump claims National Guard shooting suspect ‘...,https://nypost.com/2025/11/27/us-news/trump-cl...,107
167,2025-11-28-01-08-59 +0000,wapo,"National Guard member dies after shooting, Tru...",https://www.washingtonpost.com/dc-md-va/2025/1...,100
203,2025-11-27-20-14-05 +0000,nypost,Prosecutors will seek death penalty against Af...,https://nypost.com/2025/11/27/us-news/prosecut...,100
8,2025-11-28-14-26-55 +0000,startribune,National Guard shooting suspect was in one of ...,https://www.startribune.com/national-guard-sho...,93
31,2025-11-28-13-25-53 +0000,wapo,The National Guard shooting suspect was in one...,https://www.washingtonpost.com/nation/2025/11/...,93


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
171,107,2025-11-28-00-57-00 +0000,nypost,Trump claims National Guard shooting suspect ‘...,https://nypost.com/2025/11/27/us-news/trump-cl...
236,76,2025-11-27-17-29-02 +0000,nyt,Hong Kong Fire Shared Similarities With Grenfe...,https://www.nytimes.com/2025/11/26/world/asia/...
203,45,2025-11-27-20-14-05 +0000,nypost,Prosecutors will seek death penalty against Af...,https://nypost.com/2025/11/27/us-news/prosecut...
26,42,2025-11-28-13-45-44 +0000,nypost,Black Friday frenzy: The biggest shopping day ...,https://nypost.com/2025/11/28/us-news/black-fr...
5,33,2025-11-28-14-40-07 +0000,nyt,Israeli Forces Kill Two Palestinians in West B...,https://www.nytimes.com/2025/11/28/world/middl...
139,32,2025-11-28-05-42-45 +0000,nypost,President Trump says he’ll ‘permanently pause ...,https://nypost.com/2025/11/28/us-news/presiden...
166,31,2025-11-28-01-28-25 +0000,bbc,Carney's new Alberta oil pipeline deal already...,https://www.bbc.com/news/articles/cgkexjxmvd7o...
241,29,2025-11-27-17-00-18 +0000,bbc,'You can't see this anywhere else' - Macy's 99...,https://www.bbc.com/news/videos/cm28vgv8708o?a...
22,28,2025-11-28-13-55-04 +0000,nyt,Trump Cut Europe Out of Ukraine Talks. Here’s ...,https://www.nytimes.com/2025/11/27/world/europ...
39,27,2025-11-28-12-49-00 +0000,wsj,"Gold Futures Frozen at $4,221 as CME Outage Di...",https://www.wsj.com/finance/commodities-future...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
